In [ ]:
!pip install optuna
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
tor

In [ ]:
#!pip install datasets
from datasets import load_dataset

#import warnings
#warnings.filterwarnings("ignore")

import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")
from sklearn.metrics import auc, roc_curve, precision_score, recall_score, precision_recall_curve

import numpy as np

Setup Complete


In [ ]:
ds = load_dataset("criteo/FairJob")
df = ds['train'].to_pandas()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.14k [00:00<?, ?B/s]

fairjob.csv.gz:   0%|          | 0.00/191M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1072226 [00:00<?, ? examples/s]

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['click','user_id','impression_id','product_id'])
y = df['click']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.metrics import log_loss
import numpy as np

def evaluate_log_loss_by_class(y_true, y_proba):
    """
    Calcule la log loss globale, la log loss par classe, et la confiance moyenne sur les vrais positifs.

    Parameters
    ----------
    y_true : array-like
        Vecteur des vraies étiquettes (0 ou 1)
    y_proba : array-like, shape (n_samples, 2)
        Probabilités prédites (sorties de predict_proba)

    Returns
    -------
    dict with:
        - 'log_loss_global'
        - 'log_loss_y0'
        - 'log_loss_y1'
        - 'mean_proba_y1' (moyenne des p(y=1) quand y=1)
    """
    y_true = np.array(y_true)
    y_proba = np.array(y_proba)

    return {
        "log_loss_global": round(log_loss(y_true, y_proba), 5),
        "log_loss_y0": round(log_loss(y_true[y_true == 0], y_proba[y_true == 0], labels=[0, 1]), 5),
        "log_loss_y1": round(log_loss(y_true[y_true == 1], y_proba[y_true == 1], labels=[0, 1]), 5),
        "mean_proba_y1": round(float(y_proba[y_true == 1, 1].mean()), 5)
        }



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd

# Metrics

def click_rank_utility(df, y_pred):
    """
    Implémente la fonction Click-Rank Utility U(f).
    """
    df_test = df.loc[y_pred.index].copy()
    df_test = df_test.assign(pred=y_pred)

    # Evite le biais de position
    df_test = df_test[df_test['displayrandom'] == 1].copy()

    # Trier les produits affichés dans chaque impression par score de prédiction croissant
    df_test = df_test.sort_values(by=['impression_id', 'pred'], ascending=[True, True])

    # Calcul du rank
    df_test['rank_order'] = df_test.groupby('impression_id')['pred'].rank(method='min', ascending=True)

    # I(Y_D = 1) * rank_J f(X_J, D)
    df_test['utility'] = df_test['click'] * df_test['rank_order']

    # Moyenne de U(f) sur chaque impression’
    click_rank_per_impression = df_test.groupby('impression_id')['utility'].mean()

    # Moyenne finale sur toutes les bannières
    return click_rank_per_impression.mean()

def nllh(y_true, y_pred):
    y_pred_np = np.array(y_pred) + 1e-15
    y_true_np = np.array(y_true)

    epsilon = 1e-15

    y_pred_clip = np.clip(y_pred_np, epsilon, 1 - epsilon)

    log_loss_np = y_true_np * np.log(y_pred_clip) + (1 - y_true_np) * np.log(1 - y_pred_clip)
    return -np.mean(log_loss_np)

def DP(predictions, protected_attribute, scope):
     """
    Function to compute demographic parity with respect to scope.

    Args:
        predictions: model predictions (numpy array or pandas Series) for class 1 (probability of click)
        protected_attribute: protected attribute values (numpy array or pandas Series)
        scope: indicator for scope (numpy array or pandas Series)

    Returns:
       Demographic parity
    """

     predictions = np.array(predictions)  # Assurer que predictions est un tableau numpy
     res = 0.0  # Initialiser le résultat

    # Filtrer les prédictions où scope > 0 (par exemple, impressions valides)
     pred_group_1 = predictions[(protected_attribute > 0) & (scope > 0)]  # Groupe 1 (par exemple, groupe protégé)
     pred_group_2 = predictions[(protected_attribute <= 0) & (scope > 0)]  # Groupe 2 (par exemple, groupe non protégé)

    # Calculer la moyenne des prédictions pour chaque groupe
     mean_group_1 = np.mean(pred_group_1)
     mean_group_2 = np.mean(pred_group_2)

    # Ajouter la différence absolue au résultat
     res += np.abs(mean_group_1 - mean_group_2)

     return res


def AUC(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    return auc(fpr, tpr)

def report_results(df, y_pred_prob, y_pred):

    if not isinstance(y_pred, pd.Series):
        raise ValueError("y_pred must be a pandas Series")

    df_test = df.loc[y_pred.index].copy()
    y_true = df_test['click']
    scope = df_test['displayrandom'] == 1

    click_rank_value = click_rank_utility(df, y_pred)
    nllh_value = nllh(y_true, y_pred_prob)
    auc_value = AUC(y_true, y_pred_prob)
    dp_value = DP(y_pred_prob, df_test['protected_attribute'], scope)

    return {
    'Click Rank Utility': round(click_rank_value, 5),
    'Negative Log-Likelihood': round(nllh_value, 5),
    'AUC': round(auc_value, 5),
    'Demographic Parity': round(dp_value, 5)
}

In [ ]:
import optuna
from sklearn.preprocessing import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import log_loss
from sklearn.ensemble  import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
# Pipeline pour le pré-traitement et le modèle RandomForest
cat_cols = list(X_train.columns[4:17])  # Colonnes catégorielles
num_cols = [col for col in X_train.columns if col not in cat_cols]  # Colonnes numériques


In [ ]:
cat_pipeline = make_pipeline(
    TargetEncoder(),
    StandardScaler()
)

data_preproc = ColumnTransformer([
    ('cat_preproc', cat_pipeline, cat_cols),
    ('num_scaling', StandardScaler(), num_cols)
], remainder='passthrough', verbose_feature_names_out=False)

# Modèle RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Pipeline complet avec pré-traitement et modèle RandomForest
rf_pipe = Pipeline([
    ('preprocessing', data_preproc),
    ('rf', rf_model)
])

# Paramètres du modèle RandomForest
rf_param_dict = {
    'rf__n_estimators': 100,
    'rf__max_depth': 10,
    'rf__random_state': 42
}

rf_pipe.set_params(**rf_param_dict)
rf_pipe.fit(X_train, y_train)

# Prédictions
y_prob = rf_pipe.predict_proba(X_test)
print("\nTraining done!")


Training done!


In [ ]:
# Évaluation
results = evaluate_log_loss_by_class(y_test, y_prob)
print("Résultats de log loss :\n", results)

Résultats de log loss :
 {'log_loss_global': 0.0353, 'log_loss_y0': 0.00678, 'log_loss_y1': 4.09052, 'mean_proba_y1': 0.02813}


In [ ]:
report_results(df, y_prob[:,1], y_test)

In [ ]:
# Optimisation avec Optuna
def objective(trial):
    X_train_sub, X_val, y_train_sub, y_val = train_test_split(
        X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
    )

    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    max_depth = trial.suggest_int("max_depth", 5, 20)

    rf_pipe.set_params(
        rf__n_estimators=n_estimators,
        rf__max_depth=max_depth,
        rf__random_state=42
    )

    rf_pipe.fit(X_train_sub, y_train_sub)
    y_proba = rf_pipe.predict_proba(X_val)

    return log_loss(y_val, y_proba)

# Optimisation des hyperparamètres
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

# Application des meilleurs paramètres trouvés
best_params = study.best_trial.params

rf_pipe.set_params(
    rf__n_estimators=best_params["n_estimators"],
    rf__max_depth=best_params["max_depth"],
    rf__random_state=42
)

rf_pipe.fit(X_train, y_train)
y_prob = rf_pipe.predict_proba(X_test)

print("\nTraining done!")


results = evaluate_log_loss_by_class(y_test, y_prob)
print("Résultats de log loss :\n")
print(results)


[I 2025-05-09 12:45:33,623] A new study created in memory with name: no-name-ea39ff0a-5455-497a-b9ee-daf993e9c87c
[I 2025-05-09 12:48:11,840] Trial 0 finished with value: 0.035375227163510624 and parameters: {'n_estimators': 97, 'max_depth': 10}. Best is trial 0 with value: 0.035375227163510624.
[I 2025-05-09 12:51:47,923] Trial 1 finished with value: 0.03509750361277359 and parameters: {'n_estimators': 111, 'max_depth': 12}. Best is trial 1 with value: 0.03509750361277359.
[I 2025-05-09 12:55:47,967] Trial 2 finished with value: 0.03559512386521407 and parameters: {'n_estimators': 176, 'max_depth': 8}. Best is trial 1 with value: 0.03509750361277359.
[I 2025-05-09 13:01:24,819] Trial 3 finished with value: 0.03548297895285214 and parameters: {'n_estimators': 142, 'max_depth': 17}. Best is trial 1 with value: 0.03509750361277359.


In [ ]:
# Évaluation avec les meilleurs paramètres
results = evaluate_log_loss_by_class(y_test, y_prob)
print("Résultats de log loss avec Optuna :\n", results)

In [ ]:
report_results(df, y_prob[:,1], y_test)